# Holoviz-based plotting in the lab - Primer

This notebook gives a brief intro to using holoviz-based plotting, incl our own tools that work with it.

To make sure interactivity works correctly, run this notebook in an environment that has all the right stuff installed (i.e., your labcore environment)

## Some basic preliminaries

We assume that `raw` data is in DataDict formats.
For processing and plotting we convert to either `pandas.DataFrame` (for ungridded data) or `xarray.Dataset` (for gridded data).
This approach should be the general way we handle data in the lab.

# Brief primer on vanilla holoviz

We have some tools on top of it for general easy data exploration. 

But for data analysis it's probably very wise for everyone to learn how to use holoviz directly. 
It allows making custom plots extremely easily. 

Docs are here: https://holoviz.org; see in particular ``holoviews`` and `hvplot`.

Here is a very simple example -- we use synthetic Rabi/Chevron data for that.
But much more is possible -- check out the examples in the documentation for how to do different things.

In [6]:
# first, make a dataset in datadict
import numpy as np

from labcore.testing.dispersive_qubit_readout_data import chevron_dataset

raw_data = chevron_dataset(
    Omega_0=1e6,
    Delta_vals=np.linspace(-1e6, 1e6, 20),
    t_vals=np.linspace(0, 3e-6, 20),
    n=20,  # number of shots
)

type(raw_data), raw_data

(labcore.data.datadict.DataDict,
 signal: (400, 20)
   ⌙ repetition: (400, 20)
   ⌙ detuning (Hz): (400,)
   ⌙ time (s): (400,))

In [3]:
# next -- convert into xarray. since we know its a nice regular grid, that's the easiest choice here.
from labcore.data.datadict import datadict_to_meshgrid, dd2xr
from labcore.data.tools import split_complex

# with the holoviz (or all pydata) tools, complex data isn't always the best, so we split it immediately.
xrdata = split_complex(
    dd2xr(datadict_to_meshgrid(raw_data))
)
xrdata

<xarray.Dataset>
Dimensions:     (repetition: 20, detuning: 20, time: 20)
Coordinates:
  * repetition  (repetition) int64 1 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20
  * detuning    (detuning) float64 -1e+06 -8.947e+05 ... 8.947e+05 1e+06
  * time        (time) float64 0.0 1.579e-07 3.158e-07 ... 2.842e-06 3e-06
Data variables:
    signal_Re   (repetition, detuning, time) float64 2.085 1.837 ... 2.129 1.809
    signal_Im   (repetition, detuning, time) float64 0.4333 0.4132 ... 0.618

In [4]:
# now we can directly use hvplot on this
import hvplot.xarray  # this import patches xarray objects so we can use hvplot on them.

# plot: average over repetition, then plot the real part as 2d map.
plot = xrdata.signal_Re.mean('repetition').hvplot.quadmesh(
    x='detuning',
    y='time',
)
plot

:QuadMesh   [detuning,time]   (signal_Re)

In [5]:
# it's also pretty easy to make linecuts with sliders
_data = xrdata.mean('repetition')

# here: overlay line and scatter plot
# can change x to get a different cut
plot = _data.hvplot.line(x='time') * _data.hvplot.scatter(x='time')
plot

:DynamicMap   [detuning]
   :Overlay
      .NdOverlay.I  :NdOverlay   [Variable]
         :Curve   [time]   (value)
      .NdOverlay.II :NdOverlay   [Variable]
         :Scatter   [time]   (value)

In [7]:
# looking at readout histograms -- use simply all points in the set
# hexbin is a very simple way to have a quick look at this. 
# doesn't offer too much control, but is 
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=[],
    x='signal_Re',
    y='signal_Im',
)

:HexTiles   [signal_Re,signal_Im]

In [8]:
# if we want to select based on some time/detuning values...
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=['time', 'detuning'],
    x='signal_Re',
    y='signal_Im',
    xlim=(-3,3),
    ylim=(-3,3)
)

:DynamicMap   [time,detuning]
   :HexTiles   [signal_Re,signal_Im]